In [1]:
import pandas as pd
import ee
from collections import defaultdict
import numpy as np
import datetime
ee.Initialize()


# Parameters

In [2]:
# LOCATION OF CSV FILE with coordinates
DATA = './data.csv'

# NUMBER OF FILES TO DOWNLOAD
# set to None to download all files
DOWNLOAD_NB_FILES = 20

# RADIUS AROUND COORD IN METERS
# This is the number of meter around the point coordinate to include in the picture
RADIUS_AROUND = 400

# MULTISPECTRAL OR RGB
ALL_BANDS = True

# RANGES FOR BANDS - Only for RGB images
# Values tested on different location in nepal
# Hypothesis: Should be the same for all images
RANGE_MIN = 0
RANGE_MAX = 2000

# RANGE FOR DATES
# CSV shows one date per observation.
# However, we need to take several pictures to select ones without clouds
# We look for RANDE_DATE weeks around the date
RANDE_DATE = 2
TOI = '15/11/2016'

# GOOGLE DRIVE FOLDER
GDRIVE_FOLDER = 'SENTINEL2-PICTURES-ALL'

# Import CSV data a nd remove NaNs

In [3]:
df = pd.read_csv(DATA, index_col=0)
print(f'Len of input data: {len(df)}')

Len of input data: 1585


In [4]:
df.dropna(inplace=True)

In [5]:
print(f'{len(df)} images in the CSV file')

1433 images in the CSV file


# Google Earth Engine download


In [6]:
dataset = ee.ImageCollection('COPERNICUS/S2').select(['B4', 'B3', 'B2'])

In [7]:
def get_image_name(index, longitude, latitude):
    return f'{str(index)}_{str(longitude)}_{str(latitude)}'.replace('.', '-')

In [8]:
# Create list with a tuple per point: File name - x - y - date
coords = [(get_image_name(point[1][0], point[1][1], point[1][2]), point[1][1], point[1][2]) for point in df.loc[:, ['index', 'coord_obs_x', 'coord_obs_y']].iterrows()]

In [9]:
# File name - x - y
coords[0]

('0-0_84-28586591_27-42846051', 84.28586591, 27.42846051)

In [10]:
# Generate a rectangle containing the circle (centered on the coordinate) with radius RADIUS_AROUND
def get_geometry_radius(geometry_point):
    coord = np.array(geometry_point.getInfo()['coordinates'][0])
    return ee.Geometry.Rectangle([coord[:, 0].min(), coord[:, 1].min(), coord[:, 0].max(), coord[:, 1].max()])

# Generate the dates around the time of interest. Useful to have enough images to filter clouds
def date_range_to_collect(time_of_interest, debug=False):

    d, m, y = time_of_interest.split('/')
  
        
    target_date = datetime.date(int(y), int(m), int(d))
    delta = datetime.timedelta(weeks=RANDE_DATE)
    return target_date-delta, target_date+delta

def generate_image(image_collection, image_name, x, y, debug=True):
    if debug: print(f'Working on {image_name}: ({x}, {y})')
        
    geo = ee.Geometry.Point(x, y);
    radius = geo.buffer(RADIUS_AROUND);
    geometry_radius= get_geometry_radius(radius)
    
    spatialFiltered = image_collection.filterBounds(geo)

    date_range = date_range_to_collect(TOI, debug)
    if debug: print(f'date range: {str(date_range[0])} {str(date_range[1])}')
        
    temporalFiltered = spatialFiltered.filterDate(str(date_range[0]), str(date_range[1]))

    least_clouds = temporalFiltered.sort('CLOUD_COVER').first()


    if ALL_BANDS:
        img = least_clouds.select('B.+')
    else:
        img = least_clouds.visualize(bands=['B4', 'B3', 'B2'], min=RANGE_MIN, max=RANGE_MAX)
    
    task = ee.batch.Export.image.toDrive(img, folder=GDRIVE_FOLDER, region=geometry_radius.getInfo()['coordinates'][0], description=image_name, scale=10)
    task.start()


In [11]:
dataset_collection = ee.ImageCollection('COPERNICUS/S2')


for i, point in enumerate(coords):
    if DOWNLOAD_NB_FILES is not None and i >= DOWNLOAD_NB_FILES:
        break
    generate_image(dataset_collection, image_name=point[0], x=point[1], y=point[2])

Working on 0-0_84-28586591_27-42846051: (84.28586591, 27.42846051)
date range: 2016-11-01 2016-11-29
Working on 1-0_84-28724124_27-42850423: (84.28724124, 27.42850423)
date range: 2016-11-01 2016-11-29
Working on 2-0_84-28578211_27-42716335: (84.28578211, 27.42716335)
date range: 2016-11-01 2016-11-29
Working on 3-0_84-28731738_27-42719899: (84.28731738, 27.42719899)
date range: 2016-11-01 2016-11-29
Working on 4-0_84-40704161_27-43081959: (84.40704161, 27.43081959)
date range: 2016-11-01 2016-11-29
Working on 5-0_84-40851456_27-43078346: (84.40851456, 27.43078346)
date range: 2016-11-01 2016-11-29
Working on 6-0_84-40694692_27-42943646: (84.40694692, 27.42943646)
date range: 2016-11-01 2016-11-29
Working on 7-0_84-40851276_27-42944815: (84.40851276, 27.42944815)
date range: 2016-11-01 2016-11-29
Working on 10-0_84-43776981_27-43237146: (84.43776981, 27.43237146)
date range: 2016-11-01 2016-11-29
Working on 11-0_84-43770428_27-43235336: (84.43770428, 27.43235336)
date range: 2016-11-01